In [2]:
# Needed imports
import re
import time
import requests
import os
import pandas as pd
import html5lib
from bs4 import BeautifulSoup
import numpy as np
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException

## Scraping Code

### Function Definitions

In [ ]:
# Ceiling function
def ceil(n) :
    '''
    Calculates the ceiling of a number.
    
    Input :
        n (float) - A real number for which you want to find the ceiling value.
        
    Outpu :
        n rounded up.
    '''
    if int(n) == n :
        return n
    else :
        return int(n)+1

# Returns a number mod 40, but returns 40 instead of 0
# This is because of how the xpath for the tables on the homepage of kenpom.com is determined.
def mod_fix(n,r=40) :
    '''
    Calculates modular arithmitic, but replaces 0 with n.
    
    Input :
        n (int) - The dividend of the modular arithmitic.
        r (int) - The divisor of the modular arithmitic.
        
    Returns :
        n % r , but n instead of 0
    '''
    if n%r == 0 :
        return r
    else :
        return n%r

# Path to data
path = "../DATA/"

# Homepage for kenpom.com
home_url = "https://kenpom.com/"

def login(browser) :
    '''
    Opens kenpom.com in a chrome browser and prompts user for login information.
    Once it logs in, the function exits, leaving the browser open.
    
    Inputs :
        browser - A web browser handle controlled by python.
    '''
    # Go to webpage
    try :
        browser.get(home_url)
        # Enter user id
        try :
            # Get the email element
            usr = browser.find_element_by_name('email')
            # Empty it of contents
            usr.clear()
            # Prompt user for email
            email = str(input("EMAIL: "))
            # Type email into element
            usr.send_keys(email)
        except :
            # Should a problem occur, don't do anything
            print("Couldn't find email.")
            # Raise an error to stop the function
            raise ValueError("Field not found.")
        # Enter password
        try :
            # Get the password element
            pwd = browser.find_element_by_name('password')
            # Clear it of current contents
            pwd.clear()
            # Prompt user for password
            psswd = str(input("PASSWORD: "))
            # Type password into element
            pwd.send_keys(psswd)
        except :
            # Should a problem occur, don't do anything
            print("Couldn't find password bar.")
            # Raise an error to stop the function
            raise ValueError("Field not found.")
        # Click login button
        try :
            # Find the button
            login = browser.find_element_by_name('submit')
            # Click it
            login.click()
        except :
            # If you couldn't for some reason, do nothing
            print("Couldn't find login button.")
            # Raise an error to stop the function
            raise ValueError("Button not found.")
        # Switch to new page
        browser.switch_to_window(browser.window_handles[-1])
    except :
        # If the website doesn't exist for some reason, do nothing
        print("Couldn't find home page.")
        # Raise an error to stop the function
        raise ValueError("Page not found.")
    return browser

def get_scouting_report(browser,team_path) :
    '''
    Get the scouting report for a specific team from kenpom.com
    
    Inputs :
        browser - The browser we're controlling
        team_path (str) - The directory path to the folder where we're saving the info.
    '''
    # Get scouting report table
    # Turn the page into soup
    team_soup = BeautifulSoup(browser.page_source, 'html.parser')
    # Find the scouting report
    report = team_soup.find('div', attrs={'id':'report'})
    # Open an html file
    with open(team_path+'Scouting_Report','w') as outfile :
        # Save table to file
        outfile.write(str(report))

def get_players(browser,current_team,team_path,year) :
    '''
    Gets data for the players of a specific team in a specific year
    
    Inputs :
        browser - The browser we're controlling
        current_team (handle) - The handle for the team page
                                This is the page we want to return to when we're done.
        team_path (str) - Path to where we're saving this team's data
        year (str) - The current year we're scraping
    '''
    # The numbers in this range include all players, but some are missing different values.
    # This will skip the missing ones and grab the ones that do exist.
    for i in range(20) :
        try :
            # Player link by xpath
            player = browser.find_element_by_xpath('//*[@id="player-table"]/tbody/tr[{}]/td[2]/a[1]'.format(i))
            # Get the name of the player for the file name
            #player_name = player.text
            # Open link in new tab
            player.send_keys(Keys.COMMAND + Keys.RETURN)
            # Wait for tab to load
            time.sleep(2)
            # Switch view to new tab
            browser.find_element_by_tag_name('body').send_keys(Keys.CONTROL + Keys.TAB)
            # Switch browser to new tab
            browser.switch_to_window(browser.window_handles[-1])
            # Get player table and save to player file
            player_soup = BeautifulSoup(browser.page_source, 'html.parser')
            # Gets player name
            player_name = player_soup.find_all('div',attrs={'id' : 'content-header'})[0].find_all('span',attrs={'class' : 'name'})[0].get_text()
            # Creates the part of the path for where the player's data will be saved
            #player_path = team_path + player_name
            # Get schedule data
            table = player_soup.find_all('table', attrs={'id':'schedule-table'})
            # Check all tables if they're the ones we want
            for i in range(len(table)) :
                # This element appears above the table pertaining to the desired year's schedule
                if browser.find_element_by_xpath('//*[@id="players"]/h3[{}]'.format(i+1)).text == year + " Game Data" :
                    # Create the file as path_to_team_folder/team_name/player_name #
                    # The reson for the # at the end is to verify later that the correct table
                    #     was saved.
                    with open(team_path+player_name+' {}'.format(i),'w') as outfile :
                        # Write the file
                        outfile.write(str(table[i]))
            # Close the player tab
            browser.close()
            # Switch to team tab
            browser.switch_to_window(current_team)
        # If the xpath was for an invalid player (some of them will be) do nothing
        except NoSuchElementException :
            pass
        

def get_years(years) :
    '''
    Goes through kenpom.com year by year for the years provided to scrape data
    
    Inputs :
        years (list) - List of years we want to scrape (strings)
    '''
    # For each year of interest
    for year in years :
        # Don't want to overload the server
        time.sleep(1)
        # Get the link for given year
        try :
            # Find the year element
            link = browser.find_element_by_link_text(year)
            # Click it
            link.click()
            # Switch control to the new page
            browser.switch_to_window(browser.window_handles[-1])
            # Save page, because we'll be returning to it a lot
            current_year = browser.current_window_handle
            # Create folder for year if not already in existence
            if not os.path.exists(path+year+'/') :
                os.makedirs(path+year+'/')
            # Initialize list of teams
            teams = []
            # Counter for specific xpath
            i = 1
            # 68 teams (round of 64 and first 4 gives 4 additional)
            while len(teams) < 68 and i < 350:
                # Can't hurt to treat the server gently
                time.sleep(1)
                try :
                    # Team link by xpath
                    team = browser.find_element_by_xpath('//*[@id="ratings-table"]/tbody[{}]/tr[{}]/td[2]/a'.format(ceil(i/40),mod_fix(i)))
                    # Get name of team
                    team_name = team.text
                    # Open team link in new tab
                    team.send_keys(Keys.COMMAND + Keys.RETURN)
                    # Wait for tab to load
                    time.sleep(2)
                    # Switch to new tab
                    browser.find_element_by_tag_name('body').send_keys(Keys.CONTROL + Keys.TAB)
                    # Update browser to focus on new tab
                    browser.switch_to_window(browser.window_handles[-1])
                    # Save this team page so that it can be returned to later
                    current_team = browser.current_window_handle
                    # Set default to True, so that if the xpath exists, we don't need to change anything
                    in_tourn = True
                    # Try to find a game that was played in the tournament
                    try :
                        tournament_tag = browser.find_element_by_xpath("//*[contains(text(), 'NCAA Tournament')]")
                    except :
                        # If we failed, they weren't in the tournament, so we can ignore them
                        in_tourn = False
                    # If in the Tournament, get data
                    if in_tourn :
                        # Keep track of teams already in tournament
                        teams.append(team_name)
                        # Create path to team data
                        team_path = path+year+'/'+teams[-1]+'/'
                        # If it doesn't exist already, make a new folder
                        if not os.path.exists(team_path) :
                            os.makedirs(team_path)
                        # Grab the scouting report
                        get_scouting_report(browser,team_path)
                        # Grab player data
                        get_players(browser,current_team,team_path,year)
                    # Close tab
                    browser.close()
                    # Switch back to year tab
                    browser.switch_to_window(current_year)
                # If the specific xpath doesn't exist, don't do anything, go to the next one
                except NoSuchElementException :
                    pass
                # Keep track of how many teams we've tried.
                # As there are a finite amount of teams, we can stop once we've tried enough.
                i = i + 1
        except :
            # If there was a problem with a given year, let me know
            print("Couldn't get data for {}".format(year))

### Code to Scrape

In [ ]:
# Open chrome
browser = webdriver.Chrome('./chromedriver')
# Login
login(browser)
# Years of interest
years = ['2013', '2014','2015','2016','2017']
# Get data
get_years(years)
# Pause
time.sleep(2)
# Close browser
browser.close()

## Cleaning Code

### Clean Bad Data

In [ ]:
# Where I'm storing the data
base_path = '../DATA/2013/North Carolina A&T'
# Possible table numbers from scraper
table_numbers = ['0','1','2','3']
# Walk through scraped data
for dir_path , dir_name , file_names in os.walk(base_path) :
    for name in file_names :
        # Scouting Report formatting
        if name == 'Scouting_Report' :
            with open(os.path.join(dir_path,name),'r') as infile :
                report = infile.read()
                # Remove extra information
                report = report[332:358] + report[496:]
            # Read scouting report
            team = pds.read_html(report)[0]
            # Rename columns
            team.columns = ['Category','Offense','Defense','D-I Avg.']
            # Drop columns without data
            team = team.dropna(thresh=2)
            # Fix certain rows with unapplicable columns
            for ind in [1,24,25,27,28,29,30] :
                # Remove ranking number in offense
                team.set_value(ind,'Offense',(team.loc[ind]['Offense']).split(' ')[0])
                # Trim superfluous symbols from offense stats in these rows
                if team.loc[ind]['Offense'][0] == '+' :
                    team.set_value(ind,'Offense',team.loc[ind]['Offense'][1:])
                elif team.loc[ind]['Offense'][-1] == r'%' :
                    team.set_value(ind,'Offense',team.loc[ind]['Offense'][:-1])
                elif team.loc[ind]['Offense'][-1] == r'"' :
                    team.set_value(ind,'Offense',team.loc[ind]['Offense'][:-1])
                # Convert the string to a float
                team.set_value(ind,'Offense',float(team.loc[ind]['Offense']))
                # Remove ranking number in defense
                team.set_value(ind,'Defense',(team.loc[ind]['Defense']).split(' ')[0])
                # Trim superfluous symbols from defense stats in these rows
                if team.loc[ind]['Defense'][-1] == r'%' :
                    team.set_value(ind,'Defense',team.loc[ind]['Defense'][:-1])
                elif team.loc[ind]['Defense'][-1] == r'"' :
                    team.set_value(ind,'Defense',team.loc[ind]['Defense'][:-1])
                # The 'defense' stat should really be the D-I Avg. stat
                team.set_value(ind,'D-I Avg.',team.loc[ind]['Defense'])
                # The actual defense stat is the same as the offense stat
                team.set_value(ind,'Defense',team.loc[ind]['Offense'])
            # Fill in any not applicables with an appropriate string
            team = team.fillna('N/A')
            # Remove ranking from offense and defense values for other columns
            for ind in team.index :
                if ind not in [1,17,24,25,27,28,29,30] :
                    team.set_value(ind,'Offense',(team.loc[ind]['Offense']).split(' ')[0])
                    team.set_value(ind,'Offense',float(team.loc[ind]['Offense']))
                    team.set_value(ind,'Defense',(team.loc[ind]['Defense']).split(' ')[0])
                # 17 is the special case where they're all the same
                elif ind == 17 :
                    team.set_value(ind,'Defense',team.loc[ind]['Offense'])
                    team.set_value(ind,'D-I Avg.',team.loc[ind]['Offense'])
            # Filter to remove extra indicie columns
            team = team.filter(['Offense','Defense','D-I Avg.'])
            # Save
            team.to_csv(os.path.join(dir_path,name) + '_csv')
        elif name[-1] in table_numbers :
            with open(os.path.join(dir_path,name),'r') as infile :
                table = infile.read()
                # Remove empty superfluous <tbody> attribute
                table = table[:28] + table[44:]
            # Read in html
            player = pds.read_html(table)[0]
            # Replace unnamed columns and filter out unnecessary data
            new_columns = list(player.columns)
            new_columns[1] = 'Date'
            new_columns[5] = 'OTs'
            new_columns[7] = 'Conference'
            player.columns = new_columns
            # Change "Did not play" status to 0 values
            for ind in player.index :
                if player.loc[ind]['St'] == "Did not play" :
                    player.set_value(ind,'MP',0)
                    player.set_value(ind,'ORtg',0)
                    player.set_value(ind,'%Ps',0)
                    player.set_value(ind,'Pts',0)
                    player.loc[ind,'2Pt'] = '0-0'
                    player.loc[ind,'3Pt'] = '0-0'
                    player.loc[ind,'FT'] = '0-0'
                    player.set_value(ind,'OR',0)
                    player.set_value(ind,'DR',0)
                    player.set_value(ind,'A',0)
                    player.set_value(ind,'TO',0)
                    player.set_value(ind,'Blk',0)
                    player.set_value(ind,'Stl',0)
                    player.set_value(ind,'PF',0)
            # Filter based on important stats
            player = player.filter(['Date','Opponent','Result','OTs','Site',
                                    'Conference','MP','ORtg','%Ps','Pts','2Pt',
                                    '3Pt','FT','OR','DR','A','TO','Blk','Stl','PF'])
            # Drop NaN rows
            player = player.dropna(thresh=6)
            # Switch NaN to '0OT' in the OTs column
            player = player.fillna('0OT')
            # Switches - to 0
            player = player.replace('-',0)
            # Save
            player.to_csv(os.path.join(dir_path,name) + '_csv')

### Add Additional Data

In [ ]:
# Load DATA
path = './DATA/'

# Split function (for 2-pt, 3-pt, and ft)
def split(ratio) :
    '''
    Takes a list of number pairs separated by '-' and splits it into two lists, first and last
    
    Inputs :
        ratio (list) - A list of number pairs
        
    Outputs :
        made (list) - A list of the first numbers
        attempt (list) - A list of the last numbers
    '''
    values = [value.split('-') for value in ratio]
    made , attempt = [int(shots[0]) for shots in values], [int(shots[1]) for shots in values]
    return made, attempt

# Different types of shots
shot_types = ['2Pt','3Pt','FT']

# Walk through player files
for dir_path , dir_name , file_names in os.walk(path) :
    # List of players
    players = {}
    for name in file_names :
        # Only worry about cleaned data
        if name[-3:] == 'csv' :
            # Don't get the Scouting report
            if name[:8] != 'Scouting' :
                # Read player data
                players[name[:-6]] = pd.read_csv(os.path.join(dir_path,name))
    # Empty dict for storing team totals later
    team_values = {}
    # Get total team values
    for player in players.keys() :
        for shot_type in shot_types :
            # Split the number of made and attempted
            made , attempt = split(players[player][shot_type].values)
            # Add the number of shots player attempted to total team shots for that game
            if shot_type in team_values.keys() :
                team_values[shot_type] = [team_values[shot_type][i] + attempt[i] for i in range(len(attempt))]
            else :
                team_values[shot_type] = attempt
            # Create the percentage tab
            players[player][shot_type+' %'] = [made[i] / attempt[i] if attempt[i] != 0 else 0 for i in range(len(attempt))]
    # Add %Att for 2s, 3s, and FT
    for player in players.keys() :
        for shot_type in shot_types :
            # Split number of made and attempted
            made , attempt = split(players[player][shot_type].values)
            # Get list of percentages
            perc_att = [attempt[i] / team_values[shot_type][i] if team_values[shot_type][i] != 0 else 0 for i in range(len(attempt))]
            # Create the new column
            players[player][shot_type+' %Att'] = perc_att
        # Calculate apprx points prevented from blocks and steals
        points_prev = [2*(players[player].loc[i]['Blk'] + players[player].loc[i]['Stl'] - players[player].loc[i]['TO']) for i in players[player].index]
        # Add data to player
        players[player]['Pnts-Prev'] = points_prev
        # Get results for point margin
        res = players[player]['Result']
        # Gets 'W' for win and 'L' for loss
        result = [res[i][0] for i in range(len(res))]
        # Resets it to exclude the 'W' or 'L'
        res = [res[i][3:] for i in range(len(res))]
        # Split the scores
        score_1 , score_2 = split(res)
        # Creates the margin list
        margin = [abs(score_1[i]-score_2[i]) if result[i]=='W' else -abs(score_1[i]-score_2[i]) for i in range(len(score_1))]
        # Adds the margin column
        players[player]['Marg'] = margin
        # Refilter to remove extra indices
        players[player] = players[player].filter(['Date','Opponent','Result','OTs','Site',
                                                  'Conference','MP','ORtg','%Ps','Pts','2Pt',
                                                  '3Pt','FT','OR','DR','A','TO','Blk','Stl','PF',
                                                  '2Pt %','3Pt %','FT %','2Pt %Att','3Pt %Att',
                                                  'FT %Att','Pnts-Prev','Marg'])
        # Save new file
        players[player].to_csv(os.path.join(dir_path,player)+'_adj')

### Create Player Avgs File

The following walks through each team for every year and creates a single file that records a player and their season avgs.  It also adds a new column for each player signifying whether they are considered a "major contributer" or not (0 for yes, 1 for no).

In [3]:
path = './DATA/'

# Walk through player files
for dir_path , dir_name , file_names in os.walk(path) :
    # List of players
    players = {}
    for name in file_names :
        # Only worry about adjusted data
        if name[-3:] == 'adj' :
            # Read player data
            players[name[:-4]] = pd.read_csv(os.path.join(dir_path,name))
    # Get avgs
    cols = ['MP','ORtg','%Ps','Pts','OR','DR','A','TO','Blk','Stl','PF',
            '2Pt %','3Pt %','FT %','2Pt %Att','3Pt %Att','FT %Att',
            'Pnts-Prev','Marg','Maj Cont']
    if dir_path[-4:-1] != '201' and dir_path[-5:-1] != 'DATA' :
        # Set up empty dataframe
        avgs = pd.DataFrame(columns=cols)
        # Shrink columns because others don't have the last column (major contributor)
        cols = cols[:-1]
        for player in players.keys() :
            # Get their prominance in the tournament this year
            #     This is determined by taking their average % of possessions
            #         used during the tournament
            #     0 - 00-12%  (Benchwarmer)
            #     1 - 12-16%  (Limited role)
            #     2 - 16-20%  (Role player)
            #     3 - 20-24%  (Significant role)
            #     4 - >= 24%  (Major contributor)
            # Alternatively, you can rate their prominance as :
            #     0 - <=10%  (Practically did not contribute in tournament)
            #     1 - >10%  (Did contribute in tournament)
            # Both of these are coded, but one should be commented out.
            df = players[player]
            # The mask is for finished years only, when we can determine how much they contributed
            mask = df['Conference'] == 'NCAA-T'
            tourn_nums = df['%Ps']*mask
            tourn_games = np.count_nonzero(mask.astype(int))
            perc_poss = sum(tourn_nums)/tourn_games
            # Get numerical columns only
            data = players[player].filter(cols)
            n = len(data.index)
            # Compute avgs (non tournament)
            plyr_avgs = [sum(data[col].astype(float)*(1-mask))/n for col in cols]
            # Assign prominance by comment at beginning of for loop
            '''
            if perc_poss < 12 :
                plyr_avgs.append(0)
            elif perc_poss < 16 :
                plyr_avgs.append(1)
            elif perc_poss < 20 :
                plyr_avgs.append(2)
            elif perc_poss < 24 :
                plyr_avgs.append(3)
            else :
                plyr_avgs.append(4)
            '''
            if perc_poss >= 10 :
                plyr_avgs.append(1)
            else :
                plyr_avgs.append(0)
            avgs.loc[player] = plyr_avgs
        avgs.to_csv(dir_path+'/Player avgs')